In [1]:
import numpy
import matplotlib
matplotlib.use('Agg')
# %matplotlib
# %matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
print(sys.path)

['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/home/Seanebum/.local/lib/python3.5/site-packages', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/home/Seanebum/.local/lib/python3.5/site-packages/IPython/extensions', '/home/Seanebum/.ipython', '/mnt/c/Users/seanm/OneDrive/Documents/Research/TauHybridSolver/GillesPy2']


In [3]:
import math
import gillespy2
from gillespy2.basic_tau_hybrid_solver import BasicTauHybridSolver
from gillespy2.basic_ssa_solver import BasicSSASolver
from gillespy2.basic_tau_leaping_solver import BasicTauLeapingSolver
from gillespy2.basic_ode_solver import BasicODESolver

In [4]:

class SimpleHybridModel(gillespy2.Model):
     def __init__(self, parameter_values=None, init_v=1):
            #initialize Model
            gillespy2.Model.__init__(self, name="Simple_Hybrid_Model")

            
            #Species
            A = gillespy2.Species(name='A', initial_value=0)
            V = gillespy2.Species(name='V', initial_value=init_v)

            self.add_species([A, V])
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 20.0)
            rate2 = gillespy2.Parameter(name='rate2', expression= 10.0)
            rate_rule1 = gillespy2.RateRule(V, "cos(t)")
            self.add_parameter([rate1, rate2])
            self.add_rate_rule(rate_rule1)
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={}, products={A:1},
                   propensity_function="rate1 * V")
            
            r2 = gillespy2.Reaction(name="r2",reactants={A:1}, products={},
                    rate=rate2)
            
            self.add_reaction([r1, r2])
            self.timespan(numpy.linspace(0,100, 1001))

In [5]:
model = SimpleHybridModel()

In [6]:
%time results = model.run(solver=BasicTauHybridSolver(), show_labels=True, debug=False)
%time results2 = model.run(solver=BasicSSASolver(), show_labels=True)
# import cProfile

# cProfile.run('model.run(solver=BasicTauHybridSolver(), show_labels=True, debug=False)', sort='tot')

CPU times: user 1.47 s, sys: 31.2 ms, total: 1.5 s
Wall time: 1.45 s
CPU times: user 156 ms, sys: 15.6 ms, total: 172 ms
Wall time: 148 ms


In [7]:
plt.figure(figsize=(15,10))
plt.title("Example Simple Hybrid")
plt.xlabel("Time (s)")
plt.ylabel("Species Population")
plt.plot(results['time'], results['A'], label='A')
plt.plot(results['time'], results['V'], label='V')
plt.legend(loc='best')
plt.savefig('example_simple_hybrid.png')

In [8]:
model.plot(results2)

In [9]:
min(results['A'])

0